In [2]:
# Importing required libraries
from zipfile import ZipFile, is_zipfile
from pathlib import Path
import pandas as pd

Initial code block which main task is to unzip the contents of the compressed file.

In [3]:
# Define the path to the zipfile
zip_path = Path("data/ultimate_challenge_3.zip")


# Check if the zipfile exists with a function
def check_zipfile(zip_path: Path) -> bool:
    """Check if the zipfile exists at the given path."""
    return zip_path.exists() and is_zipfile(zip_path)


# Create a function to create a new directory
def create_target_directory(target_dir: Path):
    """Creates a new_directory and returns it."""
    target_dir.mkdir(parents=True, exist_ok=True)
    return target_dir


def extract_zipfile(zip_path: Path, target_dir: Path):
    """Extracts the contents of zip at zip_path to target_dir."""
    with ZipFile(zip_path, "r") as zip_ref:
        # get a list of all the files in the zipfile
        for file in zip_ref.namelist():
            # if the file does not exist in the target directory, extract it
            if not (target_dir / file).exists():
                zip_ref.extract(file, target_dir)
        print(f"Extracted {zip_path} to {target_dir}")


# use the functions to extract the zipfile
if check_zipfile(zip_path):
    target_dir = zip_path.parent / zip_path.stem
    create_target_directory(target_dir)
    extract_zipfile(zip_path, target_dir)

Extracted data\ultimate_challenge_3.zip to data\ultimate_challenge_3


In [4]:
# Expalnatroy Data Analysis
logins_path = "data/ultimate_challenge_3/ultimate_challenge/logins.json"

logins_df = pd.read_json(logins_path)
print(
    f"Shape of the dataframe: {logins_df.shape[0]} rows and {logins_df.shape[1]} columns"
)
logins_df.dtypes

Shape of the dataframe: 93142 rows and 1 columns


login_time    datetime64[ns]
dtype: object

In [5]:
logins_df.describe()

,login_time
count,93142
mean,1970-02-26 19:09:39.779648278
min,1970-01-01 20:12:16
25%,1970-02-04 10:37:19.750000
50%,1970-03-01 06:33:05.500000
75%,1970-03-22 04:01:10.750000
max,1970-04-13 18:57:38


In [13]:
(
    logins_df.reset_index()
    .rename(columns={"index": "original_index"})
    .sort_values("login_time")
)

,original_index,login_time
6,6,1970-01-01 20:12:16
0,0,1970-01-01 20:13:18
1,1,1970-01-01 20:16:10
3,3,1970-01-01 20:16:36
2,2,1970-01-01 20:16:37
...,...,...
93136,93136,1970-04-13 18:48:52
93137,93137,1970-04-13 18:50:19
93139,93139,1970-04-13 18:54:02
93141,93141,1970-04-13 18:54:23
